## Calibration interrogation
Examine the parameter outputs from the calibration algorithm.

In [ ]:
import numpy as np
import arviz as az

from aust_covid.inputs import get_ifrs
from aust_covid.calibration import get_priors
from emutools.utils import load_param_info
from emutools.calibration import plot_param_progression, plot_posterior_comparison, tabulate_calib_results
from aust_covid.constants import RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN
from emutools.tex import DummyTexDoc

In [ ]:
param_info = load_param_info()
parameters = param_info['value'].to_dict() | get_ifrs(DummyTexDoc())
priors = get_priors(PRIMARY_ANALYSIS in ['vacc', 'both'], param_info['abbreviations'], DummyTexDoc())
prior_names = [p.name for p in priors]
n_half_priors = round(len(priors) / 2)
first_priors = prior_names[:n_half_priors]
last_priors = prior_names[n_half_priors:]
first_third_priors = prior_names[:6]
middle_third_priors = prior_names[6: 12]
last_third_priors = prior_names[12:]

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

### Parameter traces with estimated posterior densities by chain

In [ ]:
trace_fig1 = plot_param_progression(idata, param_info['descriptions'], req_vars=first_third_priors)
trace_fig1

In [ ]:
trace_fig2 = plot_param_progression(idata, param_info['descriptions'], req_vars=middle_third_priors)
trace_fig2

In [ ]:
trace_fig3 = plot_param_progression(idata, param_info['descriptions'], req_vars=last_third_priors)
trace_fig3

### Comparison of parameter posteriors to priors

In [ ]:
comp_fig1 = plot_posterior_comparison(idata, priors, first_priors, param_info['abbreviations'].to_dict(), 0.995)
comp_fig1

In [ ]:
comp_fig2 = plot_posterior_comparison(idata, priors, last_priors, param_info['abbreviations'].to_dict(), 0.995)
comp_fig2

In [ ]:
calibration_table = tabulate_calib_results(idata, priors, param_info)
calibration_table